<h1> Predictive Maintainance MVP

Only run this notebook after you have ran Dataset_preprocess.ipynb to create a cleaned dataset.

In this notebook we train an ML model to predict whether a part is faulty or not based on sensor readings from the part. For training, we use a dataset which is adapted from a real predictive maintenance use case. The columns and rows have been anonymized, but the overall statistics of the dataset has been preserved. This will allow you to see some of the typical challenges that arise when dealing with ML for predictive maintenance.

We will go over the key aspects of a typical data science pipeline:

1. data ingest
2. data exploration <br/>
3. model training <br/>
4. model deployment <br/>

Here we train an XGBoost model, using Amazon SageMaker's built in algorithms to predict whether a part is faulty or not in assembly line for predictive maintainance. Once the faulty part is found, it will be removed from the line and taken for repair.

Instead of deploying a live endpoint here, we will deploy the model locally on the Greengrass core. But to test the model, we will test the model predictions on a test dataset and plot relevant metrics.

<h2> Import libraries and get data into dataframe

In [ ]:
import sagemaker
import numpy as np
import os
import json
import boto3

sagemaker_session = sagemaker.Session()
# bucket = 'iot-ml-predictive-maintenance' #Replace with the bucket name created for you by CloudFormation or any other bucket
# # if you are just running the notebook, not the entire lab
prefix = 'xgbdata'
LOCAL_DIR = 'training_data'
role = sagemaker.get_execution_role()
print(role)

In [ ]:
# Path to upload the trained model

bucket = sagemaker_session.default_bucket()
xgb_upload_location = os.path.join('s3://{}/{}'.format(bucket, 'xgb'))
print(xgb_upload_location)

In [ ]:
# Grab the latest XGBoost container from ECR
region = sagemaker_session.boto_region_name
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(region, 'xgboost', '0.90-1')
print(container)

In [ ]:
# The training data doesn't have any column name headers so lets include those first
with open('cols.txt', 'r') as f:
    cols = json.load(f)

In [ ]:
# Read the data directly locally
import pandas as pd

DATANAME = 'train.csv'

datorig = pd.read_csv('{}/{}'.format(LOCAL_DIR, DATANAME), names = cols)


In [ ]:
datorig.head(10)

The data consists of 2 parts:

1. Response: this is the faulty (1) or not faulty (0) signal. <br/>
2. Sensor readings: the remaining 168 columns correspond to Sensor readings.

Note the large number of 0s in the dataset. This is because not all sensors fire at the same time, and each reading only comes from a subset of the sensors. 

<h2> Data Exploration

Before building an ML model, let's go ahead and explore the dataset and look for some correlations.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### Class Imbalance
One of the first things you notice in predictive maintenance is CLASS IMBALANCE.

CLASS IMBALANCE occurs when one class (in this case faulty labels) is much less prevalent than the not faulty label.

This is a problem for ML models as this means that the model may not see enough examples of the faulty class to train accurately. Seen another way, this means that a model that always predicts a label to be not faulty is 98.4% accurate!

Our ML model will need to beat this accuracy to be useful from a business perspective. There are a number of ways for dealing with imbalanced classification problems -- upsampling, downsampling, SMOTE to name a few. For this POC we will not use these approaches and train a minimal viable model

In [ ]:
sns.countplot(datorig.Response)
print("Percent of Faulty Examples = {:.1f}".format(len(datorig[datorig['Response']==1])/len(datorig)*100))

### Correlations

Next let's extract some correlations between the Response label and the Sensor features. Since we have so many features, we can plot a heatmap for the correlations.

In [ ]:
corr = datorig.corr()

plt.figure(figsize= (10, 10))
sns.heatmap(np.abs(corr))
plt.show()

Look at the correlation heatmap above: what do you notice?

There are striking blocks of high correlations amidst relatively little correlations between other features. 
Physically these blocks could be sensors that are all firing at once on a single assembly line, or assigned to a single part. Our ML model must be able to deal with these strong and weak correlations well. A linear model may not be able to perform well when features are strongly correlated. For this reason, we choose a tree based gradient boosting model like XGBoost.

Traditionally, we might want to consider methods like PCA to perform some dimensionality reduction, but this comes at a cost of interpretability. In our factory, we want to determine which sensor is giving the anomalous signal corresponding to the faulty part, and PCA will mix the sensor signals together. 

In [ ]:
# identify what sensors correlate strongly with the faulty label
index_list = list(corr['Response'].dropna().index)
val_list = np.argsort(np.abs(corr['Response'].dropna().values))[::-1]

In [ ]:
# Get most corelated variables to the Response label. The response variable is ofcourse most correlated with itself 
#and removed
top_corrs = [index_list[x] for x in val_list[:5]]
top_corrs

In [ ]:
reduceddat = datorig[top_corrs]
reduceddat.head()

In [ ]:
# Plot how the top correlated sensor readings are correlated with each other and the Faulty and Not Faulty Labels
g = sns.PairGrid(reduceddat, hue="Response")
g.map_diag(plt.hist)
g.map_offdiag(plt.scatter)
g.add_legend()
plt.show()

<h2> Model Training and Deployment

Having explored the dataset some, we can now build a model.
Our guiding principles for Model choice are as follows:

1. There are multiple sensors with weak correlations to the final output variable <br/>
2. Outputs are highly skewed and imbalanced, so our baseline model already has 98.4% accuracy. The ML model will need to do better to drive business value <br/>
3. Sensor data may itself be correlated with each other and not all sensors may fire at once <br/>

A linear model may not work as well in this situation as the outputs are imbalanced and in real life sensor data may be highly correlated. For this reason, we choose a gradient boosting based tree model called XGBoost. 

SageMaker has a built in implementation of the XGBoost algorithm designed to work at scale. The default data type for XGBoost is LIBSVM format. But our data is in csv and XGBoost also accepts a csv input but it needs to be specified in the 'content_type' variable. We do this first.

Next we train a model: This should take less than 5 minutes and incur about 1 minute of training cost. Remember that SageMaker charges you for the time it takes the train and the infrastructure you train on. There are no upfront charges or recurring costs. Once training is complete, SageMaker tears it down automatically.

In [ ]:
# NExt we need to push the training and test data to S3
train_channel = prefix + '/train'
validation_channel = prefix + '/validation'

sagemaker_session.upload_data(path='training_data', bucket=bucket, key_prefix=train_channel)
sagemaker_session.upload_data(path='test_data', bucket=bucket, key_prefix=validation_channel)

#s3_train_data = 's3://{}/{}'.format(bucket, train_channel)
#s3_validation_data = 's3://{}/{}'.format(bucket, validation_channel)
s3_train_channel = sagemaker.session.s3_input('s3://{}/{}'.format(bucket, train_channel), content_type ='csv')
s3_valid_channel = sagemaker.session.s3_input('s3://{}/{}'.format(bucket, validation_channel), content_type ='csv')


In [ ]:
%%time
xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.c4.8xlarge',
                                    output_path=xgb_upload_location,
                                    sagemaker_session=sagemaker_session)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': s3_train_channel, 'validation': s3_valid_channel})

<h2> Model deployment and predictions

There are several options to deploy the model:

1. Live Endpoint: this approach is suitable for low latency inference against live data in the cloud. Since we want to make inferences in our factory environment, we don't choose this option. <br/>


2. Batch Transform: to test out our model accuracy, we run inferences against some test data. As our dataset is small, the test set here is just the training set + validation set. In reality, this should be completely unseen data, but this will suffice for our MVP. <br/>


3. Deploy on Greengrass: Instead of deploying a live endpoint in the AWS Cloud, we will deploy the model locally on our Greengrass core. Follow the steps in the lab to accomplish this. <br/>

<h2> Batch Transform 

Instead of deploying a live endpoint, let's run a Batch Transform job to test the model accuracy and other metrics on the entire dataset. This dataset is saved locally, so we need to first port it over to S3.


Once the dataset is in S3, we can call SageMaker Batch Transform to perform inference on the entire dataset and save the output to file

In [ ]:
# drop the Response column from the dataset.
test_set = pd.read_csv('rawdataset.csv')
resp = test_set['0']
test_set = test_set.drop(columns = ['0'])
test_set.to_csv('test.csv', index =False, header = False)

In [ ]:
test_channel = prefix + '/test'
sagemaker_session.upload_data('test.csv', bucket=bucket, key_prefix=test_channel)

Run the batch transform job. Specify the input file location and point batch transform to a path for storing the model outputs.

This might take some time again as SageMaker needs to provision the infrastructure needed to perform the batch transform, copy the model artifacts from the trained model s3 path, copy the data to perform batch inferences on and finally, run the Batch transform job.

In [ ]:
%%time
batch_input = 's3://{}/{}'.format(bucket, test_channel) # The location of the test dataset

batch_output = 's3://{}/{}/batch-inference'.format(bucket, test_channel) # The location to store the results of the batch transform job

transformer = xgb.transformer(instance_count=1, instance_type='ml.m4.xlarge', output_path=batch_output)

transformer.transform(data=batch_input, data_type='S3Prefix', content_type='text/csv', split_type='Line')

try:
    transformer.wait()
except Exception as e:
    pass

In [ ]:
# Let's download the inference into a local directory
data_dir = './data/inference'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

!aws s3 cp --recursive $transformer.output_path $data_dir

In [ ]:
y_test = pd.read_csv(os.path.join(data_dir, 'test.csv.out'), header=None)
y_vals = np.round(y_test.values)

## Metrics and Economics

Lets get some model metrics.

From a business perspective, we need to consider more than just the model accuracy but let's get that first.

Two other metrics to look at are precision and recall. How do these manifest themselves in IoT Predictive maintenance?

Precision is a measure of the number of false positives. High precision indicates low false positives. The added cost of high false positives is that the factory engineers and workers have to spend time and money troubleshooting and performing maintenance when its not required.

Imagine the other situation: if the part is actually in need of maintenance and our model predicts that it is fine. 
In IoT settings, particularly in heavy industries, a large cost is incurred when there is "unexpected downtime". Especially for mission critical equipment, this can shut down the entire site. A high recall score (low false negatives) will help minimize this as it will ensure that when a part *is* in need of maintenance, it is flagged.

An understanding of precision and recall must be tied in with the economics of the particular use case. 

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
print("Model Accuracy = {} %".format(accuracy_score(resp.values, y_vals)*100))

Recall that the baseline model was 98.4% accurate. We are getting 99.75% which beats the baseline accuracy with our XGBoost model. But the business may care more about Recall and Precision. Let's extract those next

In [ ]:
print(classification_report(resp.values, y_vals))

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots(figsize = (8, 8))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
     # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes)
    plt.tick_params(labelsize=15)  
    plt.xlabel('Predicted label', fontsize=18)
    plt.ylabel('True label',fontsize =18)
    plt.title(title, fontsize=18)
    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center", fontsize=20,
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plot_confusion_matrix(resp.values, y_vals, classes=['Normal', 'Faulty'],
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.show()

Our model is not perfect but does much better than the naive model with 98.4% accuracy. What this means is that although the model still misses some examples of faulty labels, it correctly captures a large number of them that the naive model would not be able to.

From a business standpoint, the business now has a much smaller number of recalled parts (883), at the expense of only 30 parts which the model incorrectly classified as Faulty, even though they were actually okay. It is these tradeoffs between business metrics and machine learning that requires multiple stakeholders to regularly engage in order to achieve good results.

We can now deploy this model on to Greengrass.

Go back to the predictive maintenance tutorial to continue building out this use case

# Bonus Steps

As a next step, we will use SageMaker Neo to reduce the size of the ML model we have trained. Neo minimizes models to run up to twice as fast, with no loss in accuracy. This can be crucial for deploying on edge devices such as a RaspberryPi, NVIDIA Jetson Nano etc. Here we will compile the model for deploying on an EC2 instance we're using to represent Greengrass.


As an exercise, once this model is trained, deploy this to your Greengrass Core and tag it with the Lambda function as shown in the tutorial.

In [ ]:
compiled_model = xgb
try:
    xgb.create_model()._neo_image_account(sagemaker_session.boto_region_name)
except:
    print('Neo is not currently supported in', sagemaker_session.boto_region_name)
else:
    output_path = '/'.join(xgb.output_path.split('/')[:-1]) + '/neo'
    compiled_model = xgb.compile_model(target_instance_family='ml_m4', 
                                   input_shape={'data':[1, 168]},
                                   role=role,
                                   framework='xgboost',
                                   framework_version='0.90-1',
                                   output_path=output_path)
    compiled_model.name = 'deployed-xgboost-customer-churn'
    compiled_model.image = get_image_uri(region, 'xgboost-neo', repo_version='latest')


Let's write a simple function to get the Model size in both cases

In [ ]:
def get_model_size(estimator):
    out= !aws s3 ls {estimator.model_data} --human-readable
    return out[0].split(' ')[-3]+' MB'

In [ ]:
estimators = [xgb, compiled_model] 
targets = ['Original', 'Neo for ml.m4']
locations = [e.model_data for e in estimators]
sizes = [get_model_size(e) for e in estimators]
pd.set_option('display.max_colwidth', 0)
pd.DataFrame(list(zip(targets,locations,sizes)), columns =['Targets', 'Locations','Sizes'])

Notice that SageMaker Neo reduces the model size dramatically, which can make it easier for deploying on edge devides for limited RAM and Memory.

# Thank you!